In [2]:
def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    import plotly.express as px
    import plotly.graph_objects as go
#     import plotly
    import pickle,random
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    if len(user_df)>0:
        with open('/home/jupyter/sneupane/MOODS/pickled_files/stressor_abbr.pickle', 'rb') as handle:
            stressor_lemma = pickle.load(handle)
        y_modified_labels=stressor_lemma[user_id]

        
        user_df_mod=user_df.replace(user_df.groupby('Stressor').sum().sort_values('duration', ascending=False).index[10:], 'Others').\
            groupby('Stressor').sum().sort_values(["duration"],ascending=False).reset_index()
        
        Stressors=user_df_mod.Stressor.values.tolist()
        if "Others" in Stressors:
            Stressors.append(Stressors.pop(Stressors.index('Others')))
            user_df_mod=user_df_mod.set_index('Stressor')
            user_df_mod=user_df_mod.loc[Stressors].reset_index()
            
            
        user_df_mod["Stressor_abr"]=user_df_mod["Stressor"].map(y_modified_labels)
        user_df_mod=user_df_mod.round(2)
        fig=go.Figure(data=[go.Pie(labels=user_df_mod["Stressor_abr"], values=user_df_mod["duration"],customdata=user_df_mod,
                        hole=.4,textinfo='label+percent',sort=False,direction="clockwise",showlegend=False,
                            insidetextorientation='radial',textposition="outside",
                         hovertemplate=
                            "<b>Stressors</b>: %{customdata[0][0]}<br>" +
                            "<b>Percentage</b>: %{percent}<br>" +        
                            "<extra></extra>", 
                               )])

        

        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                ),
                          font=dict(
        family="Courier New, monospace",
        size=12,
        color="Black"
    ))
        fig.update_layout(
                    autosize=True,

                    margin=dict(
                        l=10,
                        r=10,
                        b=10,
                        t=10,
                       
                    ),)
        fig.update_traces(
                  marker=dict(colors=colors, 
                              line=dict(color='#000000', width=2)))
        data_summary_user=data_summary_user_current["Prominent Stressors"]
        user_df_this_week=user_df.loc[user_df["week"]==week_num]
        if len(user_df_this_week)>0:
            user_df_this_week=user_df_this_week.groupby(["Stressor"],as_index=False)['duration'].sum()
            user_df_this_week=user_df_this_week.sort_values(["duration"],ascending=False)
            stressor_this_week=user_df_this_week["Stressor"].iloc[0]


            data_summary_user=data_summary_user.replace("_a_", stressor_this_week)
        else:
            data_summary_user=data_summary_user.replace("_a_", "No stressors reported this week")
        data_summary_user=data_summary_user.replace("week_num", str(week_num))
       
        return "Prominent Stressors",fig,data_summary_user
    else:
        return "","",""